<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/diabetic_retinopathy/AnalyzeDiabeticRetinopathyResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-4gf_ot9o/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1890 (delta 3), reused 5 (delta 0), pack-reused 1873
Receiving objects: 100% (1890/1890), 173.29 MiB | 35.94 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
!ls abstention_experiments/diabetic_retinopathy

diabeticretinopathy_useweightrescalpreds-False_labelshift-False_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-False_labelshift-True_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-True_labelshift-False_abstention_results.json.gz
diabeticretinopathy_useweightrescalpreds-True_labelshift-True_abstention_results.json.gz
flip-False_rotamt-0
flip-False_rotamt-180
flip-False_rotamt-270
flip-False_rotamt-90
flip-True_rotamt-0
flip-True_rotamt-180
flip-True_rotamt-270
flip-True_rotamt-90
RunDiabeticRetinopathyExperiments.ipynb
valid_labels.txt.gz


In [0]:
import gzip
import json

In [0]:
dr_weightrescale_false_labelshift_false_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-False_labelshift-False_abstention_results.json.gz").read())
dr_weightrescale_false_labelshift_true_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-False_labelshift-True_abstention_results.json.gz").read())

dr_weightrescale_true_labelshift_false_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-True_labelshift-False_abstention_results.json.gz").read())
dr_weightrescale_true_labelshift_true_abstention_results =\
  json.loads(gzip.open("abstention_experiments/diabetic_retinopathy/"
    +"diabeticretinopathy_useweightrescalpreds"
    +"-True_labelshift-True_abstention_results.json.gz").read())

In [20]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np

abstfracs = ['0.15', '0.3']

methodnames_noadapt = [
    'yescalib:estmarginalweightedkappa',
    'yescalib:entropy',
    'nocalib:entropy',
    'yescalib:distmaxclassprobfromone',
    'nocalib:distmaxclassprobfromone',
    'yescalib:variance',
    'nocalib:variance'    
]

methodnames_yesadapt = [
    'yesadapt:estmarginalweightedkappa',
    'noadapt:estmarginalweightedkappa',
    'yesadapt:entropy',
    'noadapt:entropy',
    'yesadapt:distmaxclassprobfromone',
    'noadapt:distmaxclassprobfromone',
    'yesadapt:variance',
    'noadapt:variance'    
]

for settingname, results_dict, methodnames in [
    ("noweightrescale_nolabelshift",
     dr_weightrescale_false_labelshift_false_abstention_results,
     methodnames_noadapt),
    ("noweightrescale_yeslabelshift",
     dr_weightrescale_false_labelshift_true_abstention_results,
     methodnames_yesadapt),
    ("yesweightrescale_nolabelshift",
     dr_weightrescale_true_labelshift_false_abstention_results,
     methodnames_noadapt),
    ("yesweightrescale_yeslabelshift",
     dr_weightrescale_true_labelshift_true_abstention_results,
     methodnames_yesadapt)]:

  print(settingname)
  abstfrac_to_methodname_to_perfs =\
    results_dict["abstfrac_to_methodname_to_perfs"]
  methodprefix_to_baseperfs = results_dict["methodprefix_to_baseperfs"]
  
  for abstfrac in abstfracs:
    
    nsamples = len(list(abstfrac_to_methodname_to_deltas[abstfrac].values())[0])
    print(abstfrac)
    
    ustats_mat = get_ustats_mat(
        method_to_perfs=abstfrac_to_methodname_to_perfs[abstfrac],
        method_names=methodnames,
        max_ustat=nsamples*(nsamples+1)/2 )
    tied_top_methods, tied_worst_methods =(
        get_tied_top_and_worst_methods(
            ustats_mat,
            methodnames,
            #0.05 threshold for one-sided test from normal approx when N=30,
            #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
            threshold=151
        ))
    tied_top_methods = [methodnames[x] for x in tied_top_methods]
    print(tied_top_methods)

noweightrescale_nolabelshift
0.15
['yescalib:estmarginalweightedkappa']
0.3
['yescalib:estmarginalweightedkappa']
noweightrescale_yeslabelshift
0.15
['yesadapt:estmarginalweightedkappa']
0.3
['yesadapt:estmarginalweightedkappa']
yesweightrescale_nolabelshift
0.15
['yescalib:estmarginalweightedkappa']
0.3
['yescalib:estmarginalweightedkappa']
yesweightrescale_yeslabelshift
0.15
['yesadapt:estmarginalweightedkappa']
0.3
['yesadapt:estmarginalweightedkappa']


In [0]:
#note: jsdiv (which, to our knowledge, has not been proposed elsewhere
# in the literature) does better than the other baselines, but not as well as
# estmarginalweightedkappa. In some cases, the uncalibrated version of
# estmarginalweightedkappa does better than the calibrated version, but
# this is likely an artifact of having access to only one pretrained model
# (unfortunately the other winners didn't make their models public, to our knowledge)